In [6]:
import langchain
print(langchain.__version__)

0.3.23


In [7]:
from langchain.agents import initialize_agent, Tool
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

In [8]:
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun

agent_search_tool = Tool(
    name="Duck Duck go Search",
    func=DuckDuckGoSearchRun().run,
    description="Use this tool to search on web if answer is not present in Knowlwdge Base of LLM"
)

agent_llm = ChatGroq(
    model="llama3-70b-8192"
)

ddg_tool = [agent_search_tool]

search_agent = initialize_agent(ddg_tool, agent_llm, agent="zero-shot-react-description", verbose=True)
agent_response = search_agent.run("What are the latest movies that will be release in April 2025")

print(agent_response)

/tmp/ipykernel_1877/4284466588.py:15: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  search_agent = initialize_agent(ddg_tool, agent_llm, agent="zero-shot-react-description", verbose=True)
/tmp/ipykernel_1877/4284466588.py:16: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent_response = search_agent.run



> Entering new AgentExecutor chain...
I need to find the latest movies that will be released in April 2025. Since this information is not present in the Knowledge Base of LLM, I will use the Duck Duck go Search tool to search for it on the web.

Action: Duck Duck go Search
Action Input: "Upcoming movies April 2025"
Observation: Here is our list of every movie releasing in theaters in April 2025. We have some huge blockbusters coming to theaters next month. ... A Minecraft Movie will be released in theaters on April 4, 2025. April 2025 will bring a dose of horror with a vampire movie and a new take on Mickey Mouse, a movie based on a popular video game, and a lot more. Horror, fantasy, and a lot more. ... Every Movie Coming To Theaters In April 2025. By Adrienne Tyler. Published Mar 28, 2025. Release Date: April 11, 2025 Coming in fourth with 12% of the vote is Warfare, a film co-directed by Ray Mendoza and Alex Garland that tells the story of American soldiers who must escape enemy t

In [9]:
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

wikipedia_search_tool = Tool(
    name="Wikipedia Search",
    func=WikipediaAPIWrapper().run,
    description="Use this tool to search on Wikipedia if answer is not present in Knowledge Base of LLM"
)
agent_llm = ChatGroq(
    model="llama3-70b-8192"
)
wiki_search_tool = [wikipedia_search_tool]
search_agent = initialize_agent(wiki_search_tool, agent_llm, agent="zero-shot-react-description", verbose=True)
agent_response = search_agent.run("Give me the information about tesla car")
print(agent_response)



> Entering new AgentExecutor chain...


Thought: I don't have enough information about Tesla cars in my Knowledge Base, so I need to search on Wikipedia.

Action: Wikipedia Search
Action Input: Tesla, Inc.
Observation: Page: Tesla, Inc.
Summary: Tesla, Inc. ( TEZ-lə or   TESS-lə) is an American multinational automotive and clean energy company. Headquartered in Austin, Texas, it designs, manufactures and sells battery electric vehicles (BEVs), stationary battery energy storage devices from home to grid-scale, solar panels and solar shingles, and related products and services.
Tesla was incorporated in July 2003 by Martin Eberhard and Marc Tarpenning as Tesla Motors. Its name is a tribute to inventor and electrical engineer Nikola Tesla. In February 2004, Elon Musk led Tesla's first funding round and became the company's chairman; in 2008, he was named chief executive officer. In 2008, the company began production of its first car model, the Roadster sports car, followed by the Model S sedan in 2012, the Model X SUV in 2015, 

In [10]:
import sqlite3
from langchain import SQLDatabase
from langchain.agents import create_sql_agent

# Connect to the SQLite database
conn = sqlite3.connect("test_database.db")
cursor = conn.cursor()
# Create a sample table and insert some data
cursor.execute('''
    CREATE TABLE IF NOT EXISTS employees (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        age INTEGER NOT NULL,
        department TEXT NOT NULL
    )
''')
cursor.execute('''
    INSERT INTO employees (name, age, department)
    VALUES
        ('Alice', 30, 'HR'),
        ('Bob', 25, 'Engineering'),
        ('Charlie', 35, 'Sales')
''')
conn.commit()
conn.close()

print("Database created and sample data inserted.")

Database created and sample data inserted.


In [11]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

agent_llm = ChatGroq(
    model="llama3-70b-8192"
)

agent_db_onnect = SQLDatabase.from_uri("sqlite:///test_database.db")
agent_db_toolkit = SQLDatabaseToolkit(db=agent_db_onnect, llm=agent_llm)
agent_db_tools = agent_db_toolkit.get_tools()

sql_db_agent = initialize_agent(agent_db_tools, agent_llm, agent="zero-shot-react-description", verbose=True)

agent_response = sql_db_agent.run("What is the name of employee who is working in HR department")
print(agent_response)



> Entering new AgentExecutor chain...
Thought: I need to find the table that contains employee information and the department they work in.

Action: sql_db_list_tables
Action Input: 
Observation: employees
Thought:I have the list of tables in the database. Now I need to check the schema of the "employees" table to see if it contains the necessary information.

Action: sql_db_schema
Action Input: employees
Observation: 
CREATE TABLE employees (
	id INTEGER, 
	name TEXT NOT NULL, 
	age INTEGER NOT NULL, 
	department TEXT NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from employees table:
id	name	age	department
1	Alice	30	HR
2	Bob	25	Engineering
3	Charlie	35	Sales
*/
Thought:I have the schema of the "employees" table, and it looks like it contains the necessary information: the employee's name and department. Now I need to craft a SQL query to retrieve the name of the employee who is working in the HR department.

Action: sql_db_query_checker
Action Input: SELECT name FROM employees WHERE d